In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from speedy_utils import *
setup_logger('D')

DEBUG    | utils_print.py:256 (setup_logger) - Logging set to DEBUG


In [3]:
import time

@memoize(cache_dir='.cache/test', cache_type='memory')
def f(s):
    # sleep for s seconds
    time.sleep(s)
    print(s)
    return s

INFO     | utils_cache.py:94 (memoize) - cache_dir: .cache/test, cache_type: memory


In [4]:
inputs = [1, 2]
outputs = multi_thread(f, inputs)

1
2


In [ ]:
class myclass:

    @memoize(cache_dir=".cache/test", cache_type="disk")
    def run(self, x, y):
        return x + y


inputs = [(1, 2), (3, 4), (5, 6)]
obj = myclass()
outputs = [obj.run(*i) for i in inputs]

INFO     | utils_cache.py:94 (memoize) - cache_dir: .cache/test, cache_type: disk
1 2
3 4
5 6


In [6]:
outputs

[3, 7, 11]